In [12]:
import openai
import json
import requests
import html
from unidecode import unidecode
from dotenv import load_dotenv
import os

load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")


In [13]:
client = openai.OpenAI(
    api_key= os.environ["OPENAI_API_KEY"]
)


In [16]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "user", "content": "Get me today's Gemini horoscope"}
    ],
    tools=[
        {
            "type": "function",
            "function": {
                "name": "get_horoscope",
                "description": "Get daily horoscope",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "zodiac_sign": {
                            "type": "string",
                            "description": "Zodiac sign, e.g., Aries, Taurus, Gemini"
                        },
                        "horoscope_type": {
                            "type": "string",
                            "enum": ["DAILY", "WEEKLY", "MONTHLY"],
                            "description": "Type of horoscope"
                        }
                    },
                    "required": ["zodiac_sign", "horoscope_type"]
                }
            }
        }
    ],
    tool_choice="auto"
)

# Save message
message = response.choices[0].message


In [ ]:
def parse_sse_json(response_text: str):
    for line in response_text.strip().splitlines():
        if line.startswith("data:"):
            json_str = line[len("data:"):].strip()
            try:
                return json.loads(json_str)
            except json.JSONDecodeError as e:
                print("❌ JSON decode error in SSE data:", e)
                return {}
    return {}


In [18]:
def invoke_tool_call(tool_call, mcp_url="http://localhost:8001/mcp/"):
    function_name = tool_call.function.name
    function_args = json.loads(tool_call.function.arguments)

    print(f"\n🔧 Tool call: {function_name} with args: {function_args}")

    # Prepare JSON-RPC payload
    mcp_payload = {
        "jsonrpc": "2.0",
        "id": 1,
        "method": "tools/call",
        "params": {
            "name": function_name,
            "arguments": function_args
        }
    }

    # Call your locally running MCP endpoint
    try:
        mcp_response = requests.post(
            url=mcp_url,
            headers={
                "Content-Type": "application/json",
                "Accept": "application/json, text/event-stream"
            },
            data=json.dumps(mcp_payload)
        )

        print(f"🔢 Status Code: {mcp_response.status_code}")
        print(f"📄 Raw Response:\n{mcp_response.text}")

        mcp_result = parse_sse_json(mcp_response.text)

    except Exception as e:
        print("❌ Error contacting MCP:", e)
        mcp_result = {
            "result": {
                "content": [{"text": "[MCP Error] Tool invocation failed."}]
            }
        }

    # Clean and return result
    result_content = mcp_result.get("result", {}).get("content", [{}])[0].get("text", "No response")
    result_content = html.unescape(result_content)
    result_content = unidecode(result_content)

    return result_content


In [ ]:
if message.tool_calls:
    for tool_call in message.tool_calls:
        final_output = invoke_tool_call(tool_call)
        print("\n✅ Tool Response:")
        print(final_output)
else:
    print("ℹ️ No tool call was triggered.")



🔧 Tool call: get_horoscope with args: {'zodiac_sign': 'Gemini', 'horoscope_type': 'DAILY'}
🔢 Status Code: 200
📄 Raw Response:
event: message
data: {"jsonrpc":"2.0","id":1,"result":{"content":[{"type":"text","text":"Horoscope for *Gemini* (DAILY) on 2025-06-14:\n\nPersonal:\nNot every love story starts perfectly. The bumps in the road are all part of the bigger plan.\n\nTravel:\nIf you have the travel bug then start researching your dream destinations. \n\nLuck:\nSome people get all the luck, huh? Donât fret - yours is coming.\n\nProfession:\nConsider downloading a spending and budget tracker to your phone and try to plan budgets. Itâs not the most exciting job, but good financial planning can go a long way.\n\nHealth:\nTry different exercises if youâve been feeling demotivated. Boxing is a great way to energise.\n\nEmotions:\nHappiness cannot be bought. Find it from within."}],"isError":false}}



✅ Tool Response:
Horoscope for *Gemini* (DAILY) on 2025-06-14:

Personal:
Not ever